##  <p style="font--family:verdana " ><a href = 'https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/'>Source  </a></p>
##  <p style="font--family:verdana " ><a href = 'https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/'>xgboost complete tuning  </a></p>

In [ ]:
import skopt

In [ ]:
from pandas import Series, DataFrame 
import xgboost as xgb 
import lightgbm as lgb
from datetime import datetime 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
data=pd.read_csv('adult.csv',header=None) 

In [ ]:
data.columns=['age','workclass','fnlwgt','education','education-num','marital_Status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','Income'] 

In [ ]:
cat_col = data.select_dtypes('object').columns.tolist()#.remove('Income')

In [ ]:
cat_col.remove('Income')

In [ ]:
cat_col

In [ ]:
data.occupation.value_counts()

In [ ]:
l=LabelEncoder() 
for c in data.columns:
    col_type = data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data[c] = Series(l.fit_transform(data[c]))
        #data[c] = data[c].astype('category')

In [ ]:
Series(l.fit_transform(data["education"])).value_counts()

In [ ]:
data.Income.value_counts() 

In [ ]:
data.info()

In [ ]:
#but not one hot encoding with light GBM
one_hot_workclass=pd.get_dummies(data.workclass) 
one_hot_education=pd.get_dummies(data.education) 
one_hot_marital_Status=pd.get_dummies(data.marital_Status) 
one_hot_occupation=pd.get_dummies(data.occupation)
one_hot_relationship=pd.get_dummies(data.relationship) 
one_hot_race=pd.get_dummies(data.race) 
one_hot_sex=pd.get_dummies(data.sex) 
one_hot_native_country=pd.get_dummies(data.native_country) 

In [ ]:
data.drop(['workclass','education','marital_Status','occupation','relationship','race','sex','native_country'],axis=1,inplace=True) 

In [ ]:
data=pd.concat([data,one_hot_workclass,one_hot_education,one_hot_marital_Status,one_hot_occupation,one_hot_relationship,one_hot_race,one_hot_sex,one_hot_native_country],axis=1)

In [ ]:
 _, i = np.unique(data.columns, return_index=True) 
data=data.iloc[:, i] 

In [ ]:
x=data.drop('Income',axis=1) 
y=data.Income 

In [ ]:
y.isnull().sum()

In [ ]:
y.fillna(y.mode()[0],inplace=True) 

In [ ]:
#Now splitting our dataset into test and train 
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

In [ ]:
#Applying xgboost
#label is used to define our outcome variable
dtrain=xgb.DMatrix(x_train, y_train)
dtest=xgb.DMatrix(x_test)

In [ ]:
parameters={'max_depth':7, 'eta':1, 'silent':1,'objective':'binary:logistic','eval_metric':'auc','learning_rate':.05, 'tree_method':'hist'}

In [ ]:
#training our model 
num_round=50
from datetime import datetime 
start = datetime.now() 
xg=xgb.train(parameters,dtrain,num_round) 
stop = datetime.now()

In [ ]:
#Execution time of the model 
execution_time_xgb = stop-start 
execution_time_xgb

In [ ]:
ypred=xg.predict(dtest) 
ypred

In [ ]:
np.unique(ypred)

In [ ]:
for i in range(0,4885): 
    if ypred[i]>=0.5:    
        ypred[i]=1 
    else: 
        ypred[i]=0  

In [ ]:
accuracy_xgb = accuracy_score(y_test,ypred) 
f1_xgb = f1_score(y_test, ypred)

In [ ]:
accuracy_xgb

## light GBM

In [ ]:
import lightgbm as lgb

In [ ]:
labels = data.columns.tolist()#.remove('Income')

In [ ]:
labels.remove('Income')

In [ ]:
labels

In [ ]:
train_data=lgb.Dataset(x_train,label=y_train, feature_name=labels, categorical_feature= cat_col)
#param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param = {'objective':'binary'}
param['metric'] = ['auc', 'binary_logloss']

In [ ]:
x_train.info()

In [ ]:
#num_round=50
start=datetime.now()
#lgbm=lgb.train(param,train_data,num_round)
lgbm=lgb.train(param,train_data)
stop=datetime.now()

In [ ]:
#Execution time of the model
execution_time_lgbm = stop-start
execution_time_lgbm

In [ ]:
#predicting on test set
ypred2=lgbm.predict(x_test)
ypred2[0:5]  # showing first 5 predictions

In [ ]:
for i in range(0,4885):
    if ypred2[i]>=.5:       # setting threshold to .5
        ypred2[i]=1
    else:  
        ypred2[i]=0

In [ ]:
#calculating accuracy/f1
#f1_lgbm = accuracy_score(ypred2,y_test)
#f1_lgbm
print(classification_report(y_test, ypred2))

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
#calculating roc_auc_score for xgboost
auc_xgb =  roc_auc_score(y_test,ypred)
auc_xgb

In [ ]:
auc_lgbm = roc_auc_score(y_test,ypred2)
comparison_dict = {'accuracy score':(f1_lgbm,f1_xgb),'auc score':(auc_lgbm,auc_xgb),'execution time':(execution_time_lgbm,execution_time_xgb)}

In [ ]:
comparison_dict

In [ ]:
#Creating a dataframe ‘comparison_df’ for comparing the performance of Lightgbm and xgb. 
comparison_df = DataFrame(comparison_dict) 
comparison_df.index= ['LightGBM','xgboost'] 
comparison_df

In [ ]:
%run Neptune.py.ipynb